In [6]:
import os
import csv
import sys
import random
import scipy
import numpy as np
from sets import Set


workspace_path = '/home/irashadow/python_workspace/Kaggle_Comp/Santander/'

In [7]:
fitems2index_read = open(workspace_path + '/dict/fitems2index_real.txt', 'r')


fitems_to_index_dict = dict()


for line in fitems2index_read:
    
    line = line.strip()
    
    #print line
    
    features_index = int(line.split('|')[0])
    feature_id = line.split('|')[1]
    feature_item = line.split('|')[2]
    
    if feature_id not in fitems_to_index_dict:
        
       fitems_to_index_dict[feature_id] = dict()
    
       
    fitems_to_index_dict[feature_id][feature_item] = features_index
       
 

fitems2index_read.close()


print fitems_to_index_dict

{'f8': {'11': 12, '10': 11, '12': 13, '-50000': 14, 'NA': 10, '1': 2, '0': 1, '3': 4, '2': 3, '5': 6, '4': 5, '7': 8, '6': 7, '9': 15, '8': 9}, 'f22': {'': 47, '03-UNIVERSITARIO': 54, '01-TOP': 53, 'NA': 55, '3': 48, '5': 50, '4': 49, '7': 52, '6': 51, '02-PARTICULARES': 56}, 'f10': {'': 16, '1': 17, '0': 23, '3': 19, '2': 18, '5': 21, '4': 20, '6': 22}, 'f5': {'27': 45, '21': 44, 'NA': 46, '1': 25, '0': 24, '3': 27, '2': 26, '5': 29, '4': 28, '7': 31, '6': 30, '9': 33, '8': 32, '11': 35, '10': 34, '13': 37, '12': 36, '15': 39, '14': 38, '17': 41, '16': 40, '19': 43, '18': 42}}


In [8]:
month_dict = dict()
month_dict['2015-01'] = 0
month_dict['2015-02'] = 1
month_dict['2015-03'] = 2
month_dict['2015-04'] = 3
month_dict['2015-05'] = 4
month_dict['2015-06'] = 5
month_dict['2015-07'] = 6
month_dict['2015-08'] = 7
month_dict['2015-09'] = 8
month_dict['2015-10'] = 9
month_dict['2015-11'] = 10
month_dict['2015-12'] = 11
month_dict['2016-01'] = 12
month_dict['2016-02'] = 13
month_dict['2016-03'] = 14
month_dict['2016-04'] = 15
month_dict['2016-05'] = 16
month_dict['2016-06'] = 17

In [9]:
def isfloat(value):
  try:
    float(value)
    return True
  except ValueError:
    return False

In [12]:
import math

data_train = open(workspace_path + 'input/train_ver2.csv')
test_train = open(workspace_path + 'input/test_ver2.csv')

input_data = [data_train]
input_data.append(test_train)


feat_write = open(workspace_path + '/feat/3_2_cust_real.feat', 'w')

line_count = 0
line_skip = 0

cust_feat_dict_tr = dict()

#holdon for featurs:f5, f8, f22(f23 in feat_inx_2)
feat_inx_1 = [5,8,10,22]
feat_inx_2 = [5,8,10,23]

feat_value_dict = dict()
feat_value_dict[5] = []
feat_value_dict[8] = []
feat_value_dict[10] = []
feat_value_dict[22] = []

#for i in range(0,10000):
    #line = data_train.readline().strip().split(',')
    
    #print line
for data_file in input_data:
    
    headline = data_file.readline().strip()
    
    for line in data_file:
        line = line.strip().split(',')

        line_count += 1    

        prt_line = ''

        if len(line) == 48 or len(line) == 24:

            cust_id = int(line[1])

            fecha_dato = line[0][:-3]

            prt_line = str(cust_id) + '|' + fecha_dato + '|'

            for feat_inx in feat_inx_1:

                line_item = line[feat_inx].replace(' ','')
                
                if line_item != '' and line_item != 'NA':

                    if feat_inx == 5:
                       line_item = int(int(line_item)/6)
                    if feat_inx == 8:
                       line_item = int(int(line_item)/20)
                    if feat_inx == 10:
                       line_item = month_dict[line_item[:-3]]-month_dict[fecha_dato]
                    if feat_inx == 22 and isfloat(line_item):
                       line_item = int(math.log(float(line_item),10))
                    
                if str(line_item) in fitems_to_index_dict['f'+str(feat_inx)]:
                   feat_inx_get = fitems_to_index_dict['f'+str(feat_inx)][str(line_item)]
                   prt_line += str(feat_inx_get) + ':1 '
                    
                   

        elif len(line) == 49 or len(line) == 25:

            cust_id = int(line[1])

            fecha_dato = line[0][:-3]

            prt_line = str(cust_id) + '|' + fecha_dato + '|'

            for feat_inx in feat_inx_2:

                line_item = line[feat_inx].replace(' ','')

                #print line

                if line_item != '' and line_item != 'NA':

                    #print 'f'+str(feat_inx)+'|'+line_item 

                    if feat_inx == 5:
                       line_item = int(int(line_item)/6)
                    if feat_inx == 8:
                       line_item = int(int(line_item)/20)
                    if feat_inx == 10:
                       line_item = month_dict[line_item[:-3]]-month_dict[fecha_dato]
                    if feat_inx == 23 and isfloat(line_item):
                       line_item = int(math.log(float(line_item),10))

                if feat_inx < 21: 
                      if str(line_item) in fitems_to_index_dict['f'+str(feat_inx)]:
                         feat_inx_get = fitems_to_index_dict['f'+str(feat_inx)][str(line_item)]
                         prt_line += str(feat_inx_get) + ':1 ' 

                else:     
                      if str(line_item) in fitems_to_index_dict['f'+str(feat_inx-1)]:
                         feat_inx_get = fitems_to_index_dict['f'+str(feat_inx-1)][str(line_item)]
                         prt_line += str(feat_inx_get) + ':1 '
        feat_write.write(prt_line[:-1] +'\n')           



feat_write.close()        
    
print(line_count)  
print(line_skip)  

14576924
0


In [13]:
import math

data_train = open(workspace_path + 'input/train_ver2.csv')
test_train = open(workspace_path + 'input/test_ver2.csv')

input_data = [data_train]
input_data.append(test_train)

line_count = 0
line_skip = 0

cust_feat_dict_tr = dict()

#holdon for featurs:f5, f8, f22(f23 in feat_inx_2)
feat_inx_1 = [5,8,10,22]
feat_inx_2 = [5,8,10,23]

feat_value_dict = dict()
feat_value_dict[5] = []
feat_value_dict[8] = []
feat_value_dict[10] = []
feat_value_dict[22] = []

for data_file in input_data:
    
    headline = data_file.readline().strip()
    
    for line in data_file:
        line = line.strip().split(',')

        line_count += 1    

        prt_line = ''

        if len(line) == 48 or len(line) == 24:

            cust_id = int(line[1])

            fecha_dato = line[0][:-3]

            prt_line = str(cust_id) + '|' + fecha_dato + '|'
            

            for feat_inx in feat_inx_1:

                line_item = line[feat_inx].replace(' ','')

                if line_item != '' and line_item != 'NA':

                    if feat_inx == 5:
                       feat_value_dict[5].append(int(line_item))
                    if feat_inx == 8:
                       feat_value_dict[8].append(int(line_item))
                    if feat_inx == 10:
                       line_item = month_dict[line_item[:-3]]-month_dict[fecha_dato]
                       feat_value_dict[10].append(line_item)
                    if feat_inx == 22 and isfloat(line_item):
                       feat_value_dict[22].append(float(line_item))

            

        elif len(line) == 49 or len(line) == 25:

            cust_id = int(line[1])

            fecha_dato = line[0][:-3]

            prt_line = str(cust_id) + '|' + fecha_dato + '|'

            for feat_inx in feat_inx_2:

                line_item = line[feat_inx].replace(' ','')

                #print line

                if line_item != '' and line_item != 'NA':

                    #print 'f'+str(feat_inx)+'|'+line_item 

                    if feat_inx == 5:
                       feat_value_dict[5].append(int(line_item))
                    if feat_inx == 8:
                       feat_value_dict[8].append(int(line_item))
                    if feat_inx == 10:
                       line_item = month_dict[line_item[:-3]]-month_dict[fecha_dato]
                       feat_value_dict[10].append(line_item)
                    if feat_inx == 23 and isfloat(line_item):
                       feat_value_dict[22].append(float(line_item))

                


        
    
print(line_count)  
print(line_skip)  
                
    
    

14576924
0


In [14]:
print '%.2f %.2f'%(max(feat_value_dict[5]), min(feat_value_dict[5]))
print '%.2f %.2f'%(max(feat_value_dict[8]), min(feat_value_dict[8]))
print '%.2f %.2f'%(max(feat_value_dict[10]), min(feat_value_dict[10]))
print '%.2f %.2f'%(max(feat_value_dict[22]), min(feat_value_dict[22]))

(f_5_max, f_5_min) = (max(feat_value_dict[5]), min(feat_value_dict[5])) 
(f_8_max, f_8_min) = (max(feat_value_dict[8]), min(feat_value_dict[8])) 
(f_10_max, f_10_min) = (max(feat_value_dict[10]), min(feat_value_dict[10])) 
(f_22_max, f_22_min) = (max(feat_value_dict[22]), min(feat_value_dict[22])) 

print f_22_min


164.00 2.00
257.00 -999999.00
6.00 0.00
28894395.51 1202.73
1202.73


In [15]:
import math

data_train = open(workspace_path + 'input/train_ver2.csv')
test_train = open(workspace_path + 'input/test_ver2.csv')

input_data = [data_train]
input_data.append(test_train)


feat_write = open(workspace_path + '/feat/3_3_cust_RealNumber.feat', 'w')

line_count = 0
line_skip = 0

cust_feat_dict_tr = dict()

#holdon for featurs:f5, f8, f22(f23 in feat_inx_2)
feat_inx_1 = [5,8,10,22]
feat_inx_2 = [5,8,10,23]

feat_10_count = 0

for data_file in input_data:
    
    headline = data_file.readline().strip()
    
    for line in data_file:
        line = line.strip().split(',')

        line_count += 1    

        prt_line = ''

        if len(line) == 48 or len(line) == 24:

            cust_id = int(line[1])

            fecha_dato = line[0][:-3]

            prt_line = str(cust_id) + '|' + fecha_dato + '|'

            for feat_inx in feat_inx_1:

                line_item = line[feat_inx].replace(' ','')
                 

                if line_item != '' and line_item != 'NA':
                    
                    if feat_inx == 5:
                       feat_value_temp = '5:%.2f '%((float(line_item) - f_5_min)/float(f_5_max - f_5_min))
                       prt_line += feat_value_temp
                      
                    if feat_inx == 8 and int(line_item) > 0:
                       feat_value_temp = '8:%.2f '%((float(line_item) - 0)/float(f_8_max))
                       prt_line += feat_value_temp
                       
                    if feat_inx == 10:
                       line_item = month_dict[line_item[:-3]]-month_dict[fecha_dato]
                       feat_value_temp = '10:%.2f '%((float(line_item) - f_10_min)/float(f_10_max - f_10_min))
                       prt_line += feat_value_temp 
                       feat_10_count += 1
                
                       
                    if feat_inx == 22 and isfloat(line_item):
                       feat_value_temp = '22:%.2f '%((float(line_item) - f_22_min)/float(f_22_max - f_22_min))
                       prt_line += feat_value_temp
                       

                
                

        elif len(line) == 49 or len(line) == 25:

            cust_id = int(line[1])

            fecha_dato = line[0][:-3]

            prt_line = str(cust_id) + '|' + fecha_dato + '|'

            for feat_inx in feat_inx_2:

                line_item = line[feat_inx].replace(' ','')

                #print line

                if line_item != '' and line_item != 'NA':

                    #print 'f'+str(feat_inx)+'|'+line_item 

                    if feat_inx == 5:
                       feat_value_temp = '5:%.2f '%((float(line_item) - f_5_min)/float(f_5_max - f_5_min))
                       prt_line += feat_value_temp
                      
                    if feat_inx == 8 and int(line_item) > 0:
                       feat_value_temp = '8:%.2f '%((float(line_item) - 0)/float(f_8_max))
                       prt_line += feat_value_temp
                    
                    if feat_inx == 10:
                       line_item = month_dict[line_item[:-3]]-month_dict[fecha_dato]
                       feat_value_temp = '10:%.2f '%((float(line_item) - f_10_min)/float(f_10_max - f_10_min))
                       prt_line += feat_value_temp 
                       feat_10_count += 1
                         
                    if feat_inx == 23 and isfloat(line_item):
                       feat_value_temp = '22:%.2f '%((float(line_item) - f_22_min)/float(f_22_max - f_22_min))
                       prt_line += feat_value_temp
                       
        
        if prt_line[-1] != '|':
              
           feat_write.write(prt_line[:-1] +'\n')    
        
        else:

           feat_write.write(prt_line +'\n')     


feat_write.close()        
    
print(line_count)  
print(line_skip)  
print(feat_10_count)
                

14576924
0
26476


In [23]:
for month_inx in range(5-4, 5):
    
    print month_inx

1
2
3
4
